In [ ]:
#!/usr/local/bin/env python
from __future__ import division
import os
import pandas as pd
from sims import utils
import numpy as np
from lmfit import Parameters, Minimizer

In [2]:
def sim(theta, model='m1', ntrials=2000, timebound=0.560, tau=.0005, si=.01, **kwargs):

      theta=fitfx.update_params(theta)
      tr=theta['tt']; mu=theta['vv']; 
      z=theta['zz']; ssd=theta['ssd']; 
      ssv=-abs(theta['ssv']); a=theta['a'];

      nSS=int(ntrials*(1-theta['pGo']))

      dx=np.sqrt(si*tau)
      Pg=0.5*(1 + mu*dx/si)
      Ps=0.5*(1 + ssv*dx/si)

      Tg=np.ceil((timebound-tr)/tau)
      Ts=np.ceil((timebound-ssd)/tau)

      trials=np.random.random_sample((ntrials, Tg))
      DVg=z + np.cumsum(np.where(trials<Pg, dx, -dx), axis=1)

      if tr<ssd:
            IXs = Tg - Ts
            init_ss = DVg[:nSS, IXs]
            init_ss[init_ss>a] = np.nan
      else:
            init_ss = np.array([z]*nSS)

      sstrials = np.random.random_sample((nSS, Ts))
      DVs = init_ss[:, None] + np.cumsum(np.where(sstrials<Ps, dx, -dx), axis=1)

      return DVg, DVs

In [ ]:
def radd_fitfx(theta, model='radd', timebound=.560, ttype='go', tau=.0005, si=.01, **kwargs):

      if ttype=='stop':
            sstrial=1
      else: 
            sstrial=0

      theta = update_params(theta)
      mlist = ['ipb', 'radd']
      if tr > ssd and sstrial: 
            t = ssd                     # start the time at ssd
      else: 
            t = tr                      # start the time at tr

      if model == 'ipa':
            a=a-z; z = 0
            ssxr = lambda x: x >= a

      elif model in mlist:
            ssxr = lambda x: x <= 0 

      ss_started = False
      no_choice_yet = True
      dx = np.sqrt(si*tau)            # step size
      e = z                           # starting point

      p = 0.5 * (1 + mu * dx / si)
      p_ss = 0.5 * (1 + ssv * dx / si)

      while no_choice_yet: 
            t += tau
      if t>=timebound and no_choice_yet:
            choice='stop'
            rt=timebound
            no_choice_yet=False
      if t>=tr:
            r=np.random.random_sample()
            if r < p:
                  e = e + dx
            else:
                  e = e - dx
            if e>=a and no_choice_yet:
                  choice='go'
                  response=1
                  rt=t
                  no_choice_yet=False

      if sstrial and t>=ssd:
            r_ss=np.random.random_sample()
            if not ss_started and model=='radd':
                  ss_started=True
                  e_ss=e
            else:
                  if r_ss < p_ss:
                          e_ss = e_ss + dx
                  else:
                        e_ss = e_ss - dx

            if ssdecision(e_ss) and no_choice_yet:
                  choice='stop'
                  rt=t
                  no_choice_yet=False
                  response=0
      if choice==ttype: 
        acc=1.00
      else: 
        acc=0.00

      return {'rt':rt, 'choice':choice, 'trial_type':ttype,
        'acc':acc, 'ssd':ssd, 'pGo':theta['pGo'], 'response':response}

In [ ]:
for model in model_list:
      
      if not hasattr(model, 'columns'):
            mstats=model.gen_stats()
            mstats['param']=mstats.index
      else:
            mstats=model
      
      slist=[int(i.split('.')[-1]) for i in mstats]

      mstats['idx']=slist
      serr=mstats.theta.sem()

mstats['mu']=mstats['theta'].values/serr